In [1]:
import cobra
from cobra import Metabolite, Gene, Reaction
from cobra.flux_analysis import single_gene_deletion
import pandas as pd
from medusa.test import load_universal_modelseed
import medusa
from pickle import load
from medusa.flux_analysis import deletion

In [2]:

with open("../results/ensembles/pto_ensemble_100_genes_added_2.pickle", 'rb') as infile:
    ensemble = load(infile)
#universal = load_universal_modelseed()

In [18]:
#set media conditions for each ensemble

#complete media has all of the exchange reactions open
complete = {}
for reaction in ensemble.base_model.reactions:
    if "EX_" in reaction.id:
        complete[reaction.id] = 1000

        
#for the other medias, use teh base biolog salts (minus heme) and add the media types needed
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)

gluc_mm = {}

for x in biolog_base_composition.ID:
    gluc_mm['EX_' + x] = 1000 
gluc_mm['EX_cpd00027_e'] = 1000

glut_mm = {}

for x in biolog_base_composition.ID:
    glut_mm['EX_' + x] = 1000
glut_mm['EX_cpd00053_e'] = 1000

#load in the anderson metabolites
anderson_df = pd.read_csv("../data/metabolite_ids_Anderson2014.csv")
anderson_media_list = []
met_ids = [met for met in anderson_df.met_id]
for met in met_ids:
    met="EX_" + str(met) + "_e"
    anderson_media_list.append(met)
    
    
anderson = {}
for x in biolog_base_composition.ID:
    anderson['EX_' + x] = 1000 
for y in anderson_media_list:
    if y in ensemble.base_model.reactions:
        anderson[y] = 1000
        #print (y + " is IN the model")
    else:
        pass
        #print (y+ " is not in the base model") 

#print (complete)        
#print ("Glucose : " , gluc_mm)        
#print ("Glutamine : " , glut_mm)        
#print ("ANDERSON : " , anderson)

{'EX_cpd00794_e': 1000, 'EX_cpd00023_e': 1000, 'EX_cpd00034_e': 1000, 'EX_cpd00041_e': 1000, 'EX_cpd00048_e': 1000, 'EX_cpd00053_e': 1000, 'EX_cpd00080_e': 1000, 'EX_cpd00107_e': 1000, 'EX_cpd00118_e': 1000, 'EX_cpd00119_e': 1000, 'EX_cpd00154_e': 1000, 'EX_cpd00156_e': 1000, 'EX_cpd00210_e': 1000, 'EX_cpd00264_e': 1000, 'EX_cpd00322_e': 1000, 'EX_cpd00637_e': 1000, 'EX_cpd00205_e': 1000, 'EX_cpd00971_e': 1000, 'EX_cpd00067_e': 1000, 'EX_cpd00060_e': 1000, 'EX_cpd00129_e': 1000, 'EX_cpd00027_e': 1000, 'EX_cpd00054_e': 1000, 'EX_cpd00009_e': 1000, 'EX_cpd00001_e': 1000, 'EX_cpd00013_e': 1000, 'EX_cpd00122_e': 1000, 'EX_cpd11596_e': 1000, 'EX_cpd01012_e': 1000, 'EX_cpd00058_e': 1000, 'EX_cpd11593_e': 1000, 'EX_cpd11592_e': 1000, 'EX_cpd11591_e': 1000, 'EX_cpd11590_e': 1000, 'EX_cpd11589_e': 1000, 'EX_cpd11588_e': 1000, 'EX_cpd01017_e': 1000, 'EX_cpd11587_e': 1000, 'EX_cpd11586_e': 1000, 'EX_cpd11585_e': 1000, 'EX_cpd11584_e': 1000, 'EX_cpd11583_e': 1000, 'EX_cpd11582_e': 1000, 'EX_cpd115

In [6]:

model_list = []
for model in ensemble.members:
    model_list.append(model.id)

EX_cpd11493_c does not seem to be an an exchange reaction. Applying bounds anyway.
EX_cpd11416_c does not seem to be an an exchange reaction. Applying bounds anyway.


In [7]:
ensemble.base_model.medium = complete
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../data/complete_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [73]:
reps = {}
for i in range(0,76):
    gene_knockouts = pd.read_excel('../data/complete_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                   , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                   , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                   , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                   , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]])
#no PST_gapfilled_77 for some reason
reps = {}
for i in range(77,99):
    gene_knockouts = pd.read_excel('../data/complete_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
rest_biolog = pd.concat([reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                   , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])
all_tables = pd.concat([all_biolog, rest_biolog])
all_tables.to_csv('../results/gene_essentiality/ensemble_essentialiity/complete_essentiality.csv')
df3 = all_tables['growth'].groupby(all_tables['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_complete.csv')

In [20]:
ensemble.base_model.medium = glut_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../data/glut_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [ ]:
reps = {}
for i in range(0,76):
    gene_knockouts = pd.read_excel('../data/glut_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                   , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                   , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                   , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                   , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]])
#no PST_gapfilled_77 for some reason
reps = {}
for i in range(77,99):
    gene_knockouts = pd.read_excel('../data/glut_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
rest_biolog = pd.concat([reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                   , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])
all_tables = pd.concat([all_biolog, rest_biolog])
all_tables.to_csv('../results/gene_essentiality/ensemble_essentialiity/glut_essentiality.csv')
df3 = all_tables['growth'].groupby(all_tables['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_glut.csv')

In [21]:
ensemble.base_model.medium = gluc_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../data/gluc_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        value.to_excel(writer, sheet_name=key)


performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [ ]:
reps = {}
for i in range(0,76):
    gene_knockouts = pd.read_excel('../data/gluc_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                   , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                   , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                   , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                   , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]])
#no PST_gapfilled_77 for some reason
reps = {}
for i in range(77,99):
    gene_knockouts = pd.read_excel('../data/gluc_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
rest_biolog = pd.concat([reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                   , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])
all_tables = pd.concat([all_biolog, rest_biolog])
all_tables.to_csv('../results/gene_essentiality/ensemble_essentialiity/gluc_essentiality.csv')
df3 = all_tables['growth'].groupby(all_tables['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_gluc.csv')

In [22]:
ensemble.base_model.medium = anderson
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../data/anderson_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [ ]:
reps = {}
for i in range(0,76):
    gene_knockouts = pd.read_excel('../data/anderson_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                   , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                   , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                   , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                   , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]])
#no PST_gapfilled_77 for some reason
reps = {}
for i in range(77,99):
    gene_knockouts = pd.read_excel('../data/anderson_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
rest_biolog = pd.concat([reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                   , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])
all_tables = pd.concat([all_biolog, rest_biolog])
all_tables.to_csv('../results/gene_essentiality/ensemble_essentialiity/anderson_essentiality.csv')
df3 = all_tables['growth'].groupby(all_tables['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_anderson.csv')